<a href="https://colab.research.google.com/github/KorStats/BIGDATAGISA/blob/main/%EA%B3%BC%EC%A0%95/(%EB%85%B8%EC%9D%B4)_%EC%9B%90%EB%B3%B8I%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A4%91%EB%B6%84%EB%A5%98_%ED%95%99%EC%8A%B5%EA%B2%B0%EA%B3%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

     |████████████████████████████████| 47.3 MB 79.7 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.1 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595734 sha256=340d513bfdafa54145d46dbf1e63fea4c8b6d56892233c657c96a57e29cd8d4b
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.0 MB/s 
     |████████████████████████████████| 769 kB 5.0 MB/s 
     |████████████████████████████████| 895 kB 46.0 MB/s 
     |████████████████████████████████| 3.0 MB 28.4 MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-7sns1bp5
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/p

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 기본   산업분류자동화	'Colab Notebooks'  'My Drive'


In [10]:
import pandas as pd
df=pd.read_csv('/mydrive/산업분류자동화/1. 실습용자료.txt', sep = "|", encoding = "euc-kr")
ts=pd.read_csv('/mydrive/산업분류자동화/답안 작성용 파일.csv', encoding = "euc-kr")

In [11]:
df=df[df['digit_1']=='I']
ts=ts[ts['digit_1']=='I']

In [13]:
df=df.reset_index(drop=True)

## 데이터 전처리

In [14]:
df.head(10)

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000007,I,56,561,음식점에서,접객시설을 갖추고,참치회(일본식)
1,id_0000012,I,56,561,음식점,접객시설을 갖추고,냉면
2,id_0000014,I,56,561,음식점에서,접객시설을 갖추고,회
3,id_0000023,I,56,562,유흥주점에서,무도시설을 갖추고,술판매
4,id_0000025,I,56,561,음료점에서,"빙수, 아이스크림, 커피, 디저트 제공",커피.빙수등을제공
5,id_0000026,I,56,561,영업점에서,일반소비자에게,오징어빵판매
6,id_0000033,I,56,561,음식점에서,접객시설을 갖추고,소갈비살
7,id_0000035,I,56,561,음식점,접객시설을갖추고,치킨
8,id_0000036,I,56,561,음식점에서,접객 시설을 갖추고,중화요리
9,id_0000038,I,56,562,카페에서,접객시설을갖추고,"커피,음료수"


In [15]:
df['text']=df['text_obj'].map(str)+" "+df['text_mthd'].map(str)+" "+df['text_deal'].map(str)
i=[]
for sentence in df['text']:
    a=sentence.replace('nan', '')
    i.append(a)
df['text']=pd.DataFrame(i)
df=df.drop(['text_obj','text_mthd', 'text_deal'], axis=1) # 기존에 있던 세개의 text변수 삭제
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000007,I,56,561,음식점에서 접객시설을 갖추고 참치회(일본식)
1,id_0000012,I,56,561,음식점 접객시설을 갖추고 냉면
2,id_0000014,I,56,561,음식점에서 접객시설을 갖추고 회
3,id_0000023,I,56,562,유흥주점에서 무도시설을 갖추고 술판매
4,id_0000025,I,56,561,"음료점에서 빙수, 아이스크림, 커피, 디저트 제공 커피.빙수등을제공"
...,...,...,...,...,...
187420,id_0999978,I,56,561,사업장구내식당에서 회사와 계약을 맺고 음식제공
187421,id_0999981,I,56,562,"주점에서 접객시설갖추고 주류,안주류"
187422,id_0999987,I,56,561,음식점에서 육류 제공
187423,id_0999988,I,55,551,펜션운영업


In [16]:
# 결측값 확인
df.isnull().sum()

AI_id      0
digit_1    0
digit_2    0
digit_3    0
text       0
dtype: int64

In [17]:
df.shape

(187425, 5)

In [18]:
df['digit_2'].value_counts()

56    173459
55     13966
Name: digit_2, dtype: int64

In [19]:
df['digit_3'].value_counts()

561    123703
562     49756
551     12549
559      1417
Name: digit_3, dtype: int64



#### **추후 과제: 분류가 현재 매우 불균형함 / 정규화를 이용해서 균형을 맞추면 정확도가 올라가는지 확인 필요**


In [20]:
df.loc[(df['digit_3'] == 551), 'digit_3'] = 0  
df.loc[(df['digit_3'] == 559), 'digit_3'] = 1  
df.loc[(df['digit_3'] == 561), 'digit_3'] = 2  
df.loc[(df['digit_3'] == 562), 'digit_3'] = 3  

In [21]:
data_list = []
for q, label in zip(df['text'], df['digit_3'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

print(data_list[:5])

[['음식점에서 접객시설을 갖추고 참치회(일본식)', '2'], ['음식점 접객시설을 갖추고 냉면', '2'], ['음식점에서 접객시설을 갖추고 회', '2'], ['유흥주점에서 무도시설을 갖추고 술판매', '3'], ['음료점에서 빙수, 아이스크림, 커피, 디저트 제공 커피.빙수등을제공', '2']]


## Train data & test data

In [22]:
from sklearn.model_selection import train_test_split             
dataset_train, dataset_test = train_test_split(data_list, test_size=0.1, random_state=0) #추후에 test_size 수정 필요

# KoBERT 입력 데이터로 만들기

In [23]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [24]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5 #추후 num_epochs 증가할 필요 있음
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

 ### 추후 과제 : 쓰이는 parameters 최적화 필요

In [25]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [26]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [27]:
data_train[0]

(array([   2, 4086, 5370, 6712, 7088, 5353, 7461, 5439, 2861, 6897,  517,
        5796, 6314, 6642,    3,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(15, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 2)

In [28]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# KoBERT 학습 모델 만들기

In [29]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=4,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [30]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


# KoBERT 모델 학습시키기

In [31]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/2636 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.515560269355774 train acc 0.125
epoch 1 batch id 201 loss 0.2860580086708069 train acc 0.663945895522388
epoch 1 batch id 401 loss 0.036390017718076706 train acc 0.8175264962593516
epoch 1 batch id 601 loss 0.19528397917747498 train acc 0.8705802828618968
epoch 1 batch id 801 loss 0.010952271521091461 train acc 0.8985642946317104
epoch 1 batch id 1001 loss 0.18002574145793915 train acc 0.9152253996003996
epoch 1 batch id 1201 loss 0.0637260153889656 train acc 0.9262333472106578
epoch 1 batch id 1401 loss 0.00523512065410614 train acc 0.9341876338329764
epoch 1 batch id 1601 loss 0.09870846569538116 train acc 0.9402814647095565
epoch 1 batch id 1801 loss 0.3209141492843628 train acc 0.9452647834536368
epoch 1 batch id 2001 loss 0.07366074621677399 train acc 0.9492831709145427
epoch 1 batch id 2201 loss 0.015963422134518623 train acc 0.9524789868241709
epoch 1 batch id 2401 loss 0.03425327688455582 train acc 0.9553701582673886
epoch 1 batch id 2601 loss 0.008614

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.9864547781569966


  0%|          | 0/2636 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.084964819252491 train acc 0.984375
epoch 5 batch id 201 loss 0.058458469808101654 train acc 0.9881840796019901
epoch 5 batch id 401 loss 0.008280927315354347 train acc 0.9886611596009975
epoch 5 batch id 601 loss 0.021820515394210815 train acc 0.9886907237936772
epoch 5 batch id 801 loss 0.006079388316720724 train acc 0.9887835518102372
epoch 5 batch id 1001 loss 0.17294159531593323 train acc 0.9886831918081919
epoch 5 batch id 1201 loss 0.07373175024986267 train acc 0.9885251873438801
epoch 5 batch id 1401 loss 0.008481822907924652 train acc 0.9885907387580299
epoch 5 batch id 1601 loss 0.025448480620980263 train acc 0.9884056839475328
epoch 5 batch id 1801 loss 0.24119748175144196 train acc 0.9885653803442532
epoch 5 batch id 2001 loss 0.0770619809627533 train acc 0.988677536231884
epoch 5 batch id 2201 loss 0.0070009781047701836 train acc 0.9887267151294866
epoch 5 batch id 2401 loss 0.03262677416205406 train acc 0.9889303935860059
epoch 5 batch id 2601 los

  0%|          | 0/293 [00:00<?, ?it/s]

epoch 5 test acc 0.986294795221843


In [32]:
#학습 모델 저장
torch.save(model, 'I_ori_so_model.pt')